In [1]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 31.5 MB/s 
     |████████████████████████████████| 6.6 MB 59.3 MB/s 
     |████████████████████████████████| 120 kB 72.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.7 MB/s 
     |████████████████████████████████| 212 kB 49.1 MB/s 
     |████████████████████████████████| 115 kB 12.7 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 28.4 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [3]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/新建文件夹/ERAI Dataset_with_translation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/新建文件夹/ERAI Dataset_with_translation


In [8]:
pd.set_option('display.max_rows', None)

In [9]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df.reset_index(drop = True,inplace = True)
df = df.dropna(axis = 0)



df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))


In [10]:
PRETRAINED_MODEL_NAME =  "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [11]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.float),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.float),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [12]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)
    

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    cls_output = []
    sent1_output = []
    sent2_output = []

    for last_output,len1,len2 in zip(output[0],length1,length2):
      
      
      sent1_output.append(torch.mean(last_output[1:(len1+1)],dim=0))
      sent2_output.append(torch.mean(last_output[len1+2:(len1+2+len2)],dim=0))

    cls_output = output[1]
    sent1_output = torch.stack(sent1_output)
    
    sent2_output = torch.stack(sent2_output)
    
    
    cls_output = self.out_cls(cls_output)
    
    
    
    return cls_output,self.out_reg(sent1_output),self.out_reg(sent2_output)

In [13]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [14]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [15]:
EPOCHS = 15

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.BCEWithLogitsLoss().to(device)


In [16]:
def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  t_examples,
  v_examples,
  mpp_flag,
  model_name
  ):
  model.train()
  max_len = 256

  global acccuracy_state
  
  for epo in range(epoch):
    
    train_loss = []
    
    validation_loss = []
    
    

    train_prediction = []
    train_real_data = []
    test_prediction = []
    test_real_data = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
     
      if mpp_flag == True:
        loss1 = loss_cls(output_cls.view(-1),mpp_targets).to(device)
        
      else:
        loss1 = loss_cls(output_cls.view(-1),ml_targets).to(device)
        

     
      loss = loss1

      train_loss.append(loss.item())
      
      for i in output_cls:
        if i<0:
          train_prediction.append(0)
        else:
          train_prediction.append(1)
      
      if mpp_flag == True:
        for i in mpp_targets:
          train_real_data.append(i.cpu().item())
      else:
        for i in ml_targets:
          train_real_data.append(i.cpu().item())


      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
        if mpp_flag == True:
          loss1 = loss_cls(output_cls.view(-1),mpp_targets).to(device)
          
        else:
          loss1 = loss_cls(output_cls.view(-1),ml_targets).to(device)
          
        
        
        loss = loss1
        validation_loss.append(loss.item())

        for i in output_cls:
          if i.item()<0:
            test_prediction.append(0)
          else:
            test_prediction.append(1)
          
        if mpp_flag == True:
          for i in mpp_targets:
            test_real_data.append(i.cpu().item())
        else:
          for i in ml_targets:
            test_real_data.append(i.cpu().item())


   
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)
    
    validation_accuracy = accuracy_score(test_real_data,test_prediction)
    train_accuracy = accuracy_score(train_real_data,train_prediction)

    if(validation_accuracy > acccuracy_state):
      acccuracy_state = validation_accuracy
      if mpp_flag == True:
        torch.save(model.state_dict(), model_name+'mpp_singe_value_state.bin')
      if mpp_flag == False:
        torch.save(model.state_dict(), model_name+'ml_singe_value_state.bin')


    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_accuracy, final_validation_loss,validation_accuracy))

In [17]:
def balance_dataset(dataset,mpp_flag):
  new_dataset = deepcopy(dataset)
  if mpp_flag == True:

    label_difference = new_dataset['MPP_label'].value_counts()[1] - new_dataset['MPP_label'].value_counts()[0]
    if label_difference == 0:
      return dataset
    if label_difference < 0 :
      label_difference = new_dataset['MPP_label'].value_counts()[0] - new_dataset['MPP_label'].value_counts()[1]
      condition = new_dataset['MPP_label'] == 0
    if label_difference > 0:
      condition = new_dataset['MPP_label'] == 1
    subset = new_dataset[condition].sample(n = label_difference)
    subset[['post1', 'post2']] = subset[['post2', 'post1']]
    subset[['post1_en', 'post2_en']] = subset[['post2_en', 'post1_en']]
    subset[['post1_en', 'post2_en']] = subset[['post2_en', 'post1_en']]
    subset[['MPP1', 'MPP2']] = subset[['MPP2', 'MPP1']]
    subset[['ML1', 'MPP2']] = subset[['ML2', 'MPP1']]
    subset['ML_label'] = subset['ML_label'].apply(lambda x: swich_value(x))
    subset['MPP_label'] = subset['MPP_label'].apply(lambda x: 0)
    dataset = pd.concat([new_dataset,subset])

  if mpp_flag == False:
    label_difference = new_dataset['ML_label'].value_counts()[1] - new_dataset['ML_label'].value_counts()[0]

    if label_difference == 0:
      return dataset
    if label_difference < 0 :
      label_difference = new_dataset['ML_label'].value_counts()[0] - new_dataset['ML_label'].value_counts()[1]
      condition = new_dataset['ML_label'] == 0
    else:

      condition = new_dataset['M_label'] == 1

    subset = new_dataset[condition].sample(n = label_difference)  
    subset[['post1', 'post2']] = subset[['post2', 'post1']]
    subset[['post1_en', 'post2_en']] = subset[['post2_en', 'post1_en']]
    subset[['post1_en', 'post2_en']] = subset[['post2_en', 'post1_en']]
    subset[['MPP1', 'MPP2']] = subset[['MPP2', 'MPP1']]
    subset[['ML1', 'MPP2']] = subset[['ML2', 'MPP1']]
    subset['MPP_label'] = subset['MPP_label'].apply(lambda x: swich_value(x))
    subset['ML_label'] = subset['ML_label'].apply(lambda x: 0)
    dataset = pd.concat([new_dataset,subset])

  return dataset


In [18]:
def swich_value(x):
  if x == 0:
    x = 1
  else:
    x = 0
  return x

In [20]:
random_seed = [171,354,550,667,985]
model_name = ['roberta','chi','fin','sec']

for i in model_name:
  
  

  if i == 'chi':
    PRETRAINED_MODEL_NAME = "bert-base-chinese"
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'roberta':
    PRETRAINED_MODEL_NAME = '/content/drive/MyDrive/ROBERT_4_model.bin'
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
  if i == 'fin':
    PRETRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'sec':
    PRETRAINED_MODEL_NAME = "nlpaueb/sec-bert-shape"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

  acccuracy_state = 0



  for j in random_seed:
    df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
    df = shuffle(df,random_state=j)

    
    train = df[0:156]
    
    train = balance_dataset(train,mpp_flag = True)
    print('The length of train set '+str(len(train)))

    test = df[156:200]
    test = balance_dataset(test,mpp_flag = True)
    print('The length of test set '+str(len(test)))

    if i=='sec'or i=='fin':

      train_post1 = train['post1_en']
      train_post2 = train['post2_en']
      test_post1 = test['post1_en']
      test_post2 = test['post2_en']
    else:
      train_post1 = train['post1']
      train_post2 = train['post2']
      test_post1 = test['post1']
      test_post2 = test['post2']

    train_mpp_label = train['MPP_label']
    train_ml_label = train['ML_label']
    train_mpp_value1 = train['MPP1'] 
    train_mpp_value2 = train['MPP2']
    train_ml_value1 = train['ML1']
    train_ml_value2 = train['ML2']

    
    test_mpp_label = test['MPP_label']
    test_ml_label = test['ML_label']
    test_mpp_value1 = test['MPP1'] 
    test_mpp_value2 = test['MPP2']
    test_ml_value1 = test['ML1']
    test_ml_value2 = test['ML2']

    max_len = 256
    batch_size = 16
    train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
    test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

    label_num_cls = 1
    label_num_reg = 1
    # model_mpp= BertNerClassifier(label_num_cls,label_num_reg).to(device)
    model_ml= BertNerClassifier(label_num_cls,label_num_reg).to(device)
    # optimizer_mpp = AdamW(model_mpp.parameters(), lr=2e-6, correct_bias=False)
    optimizer_ml = AdamW(model_ml.parameters(), lr=2e-6, correct_bias=False)
    
    mpp_flag = False
    train_and_eval(model_ml, 
      train_dataloader, 
      test_dataloader, 
      optimizer_ml,  
      device,
      EPOCHS,
      len(train),
      len(test),
      mpp_flag,
      i
      )
    # mpp_flag = False
    # train_and_eval(model_ml, 
    #   train_dataloader, 
    #   test_dataloader, 
    #   optimizer_ml,  
    #   device,
    #   EPOCHS,
    #   len(train),
    #   len(test),
    #   mpp_flag,
    #   i
    #   )




The length of train set 168
The length of test set 50


Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 0.7002, Train MPP Accuracy: 0.5238, Valid Total Loss: 0.7044, Valid MPP Accuracy: 0.5400
Epoch [2] Train Totol Loss: 0.6992, Train MPP Accuracy: 0.4762, Valid Total Loss: 0.6987, Valid MPP Accuracy: 0.5400
Epoch [3] Train Totol Loss: 0.6787, Train MPP Accuracy: 0.7083, Valid Total Loss: 0.6979, Valid MPP Accuracy: 0.5200
Epoch [4] Train Totol Loss: 0.6659, Train MPP Accuracy: 0.6607, Valid Total Loss: 0.6998, Valid MPP Accuracy: 0.4600
Epoch [5] Train Totol Loss: 0.6352, Train MPP Accuracy: 0.7440, Valid Total Loss: 0.7089, Valid MPP Accuracy: 0.4800
Epoch [6] Train Totol Loss: 0.5839, Train MPP Accuracy: 0.7738, Valid Total Loss: 0.7314, Valid MPP Accuracy: 0.4600
Epoch [7] Train Totol Loss: 0.4963, Train MPP Accuracy: 0.8750, Valid Total Loss: 0.7824, Valid MPP Accuracy: 0.4600
Epoch [8] Train Totol Loss: 0.3843, Train MPP Accuracy: 0.9107, Valid Total Loss: 0.8542, Valid MPP Accuracy: 0.4600
Epoch [9] Train Totol Loss: 0.2806, Train MPP Accuracy: 0.9226, 

Some weights of the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/ROBERT_4_model.bin and are newly initialized: ['bert.pooler.

Epoch [1] Train Totol Loss: 0.7028, Train MPP Accuracy: 0.5341, Valid Total Loss: 0.6936, Valid MPP Accuracy: 0.5217
Epoch [2] Train Totol Loss: 0.6846, Train MPP Accuracy: 0.5682, Valid Total Loss: 0.6957, Valid MPP Accuracy: 0.5000
Epoch [3] Train Totol Loss: 0.6753, Train MPP Accuracy: 0.5568, Valid Total Loss: 0.6960, Valid MPP Accuracy: 0.5000
Epoch [4] Train Totol Loss: 0.6594, Train MPP Accuracy: 0.5682, Valid Total Loss: 0.6972, Valid MPP Accuracy: 0.4783
Epoch [5] Train Totol Loss: 0.6315, Train MPP Accuracy: 0.6534, Valid Total Loss: 0.7074, Valid MPP Accuracy: 0.4565
Epoch [6] Train Totol Loss: 0.5719, Train MPP Accuracy: 0.7443, Valid Total Loss: 0.7328, Valid MPP Accuracy: 0.4348
Epoch [7] Train Totol Loss: 0.4765, Train MPP Accuracy: 0.8295, Valid Total Loss: 0.7768, Valid MPP Accuracy: 0.4565
Epoch [8] Train Totol Loss: 0.3733, Train MPP Accuracy: 0.8409, Valid Total Loss: 0.8170, Valid MPP Accuracy: 0.3696
Epoch [9] Train Totol Loss: 0.2934, Train MPP Accuracy: 0.8409, 

...

Epoch [1] Train Totol Loss: 0.7064, Train MPP Accuracy: 0.4872, Valid Total Loss: 0.6913, Valid MPP Accuracy: 0.5323
Epoch [2] Train Totol Loss: 0.6680, Train MPP Accuracy: 0.5641, Valid Total Loss: 0.6906, Valid MPP Accuracy: 0.5484
Epoch [3] Train Totol Loss: 0.6192, Train MPP Accuracy: 0.6667, Valid Total Loss: 0.6969, Valid MPP Accuracy: 0.4839
Epoch [4] Train Totol Loss: 0.5552, Train MPP Accuracy: 0.8846, Valid Total Loss: 0.7072, Valid MPP Accuracy: 0.4677
Epoch [5] Train Totol Loss: 0.4587, Train MPP Accuracy: 0.9423, Valid Total Loss: 0.7259, Valid MPP Accuracy: 0.4839
Epoch [6] Train Totol Loss: 0.3372, Train MPP Accuracy: 0.9808, Valid Total Loss: 0.7621, Valid MPP Accuracy: 0.5000
Epoch [7] Train Totol Loss: 0.2230, Train MPP Accuracy: 0.9936, Valid Total Loss: 0.8165, Valid MPP Accuracy: 0.5161
Epoch [8] Train Totol Loss: 0.1410, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8870, Valid MPP Accuracy: 0.5323
Epoch [9] Train Totol Loss: 0.0904, Train MPP Accuracy: 1.0000, 

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.6980, Train MPP Accuracy: 0.4940, Valid Total Loss: 0.6792, Valid MPP Accuracy: 0.5385
Epoch [2] Train Totol Loss: 0.6652, Train MPP Accuracy: 0.6928, Valid Total Loss: 0.6840, Valid MPP Accuracy: 0.4615
Epoch [3] Train Totol Loss: 0.6184, Train MPP Accuracy: 0.7590, Valid Total Loss: 0.6825, Valid MPP Accuracy: 0.4808
Epoch [4] Train Totol Loss: 0.5447, Train MPP Accuracy: 0.7711, Valid Total Loss: 0.6811, Valid MPP Accuracy: 0.5192
Epoch [5] Train Totol Loss: 0.4420, Train MPP Accuracy: 0.8735, Valid Total Loss: 0.7041, Valid MPP Accuracy: 0.5000
Epoch [6] Train Totol Loss: 0.3507, Train MPP Accuracy: 0.8916, Valid Total Loss: 0.7091, Valid MPP Accuracy: 0.5192
Epoch [7] Train Totol Loss: 0.2833, Train MPP Accuracy: 0.9157, Valid Total Loss: 0.7225, Valid MPP Accuracy: 0.5192
Epoch [8] Train Totol Loss: 0.2445, Train MPP Accuracy: 0.9036, Valid Total Loss: 0.7586, Valid MPP Accuracy: 0.5192
Epoch [9] Train Totol Loss: 0.2195, Train MPP Accuracy: 0.9217, 

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.6956, Train MPP Accuracy: 0.5284, Valid Total Loss: 0.7158, Valid MPP Accuracy: 0.5000
Epoch [2] Train Totol Loss: 0.6600, Train MPP Accuracy: 0.5739, Valid Total Loss: 0.7278, Valid MPP Accuracy: 0.4348
Epoch [3] Train Totol Loss: 0.6140, Train MPP Accuracy: 0.6932, Valid Total Loss: 0.7436, Valid MPP Accuracy: 0.4783
Epoch [4] Train Totol Loss: 0.5567, Train MPP Accuracy: 0.7386, Valid Total Loss: 0.7875, Valid MPP Accuracy: 0.4783
Epoch [5] Train Totol Loss: 0.4907, Train MPP Accuracy: 0.7841, Valid Total Loss: 0.8505, Valid MPP Accuracy: 0.4565
Epoch [6] Train Totol Loss: 0.4335, Train MPP Accuracy: 0.8011, Valid Total Loss: 0.9000, Valid MPP Accuracy: 0.4565
Epoch [7] Train Totol Loss: 0.3801, Train MPP Accuracy: 0.8352, Valid Total Loss: 0.9537, Valid MPP Accuracy: 0.4348
Epoch [8] Train Totol Loss: 0.3450, Train MPP Accuracy: 0.8523, Valid Total Loss: 1.0108, Valid MPP Accuracy: 0.4348
Epoch [9] Train Totol Loss: 0.3182, Train MPP Accuracy: 0.8523, 

In [ ]:
def generate_test_data(post1,post2,max_len):
  
  
  
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

In [ ]:
 
class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']

      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.first_sen_len[index],self.second_sen_len[index]

In [ ]:
def generate_test_dataloader(post1,post2,max_len,batch_size):
  data = generate_test_data(post1,post2,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader

In [ ]:
max_len = 256
batch_size = 16
test_post1 = df2['post1']
test_post2 = df2['post2']
   
test_dataloader = generate_test_dataloader(test_post1,test_post2,max_len,batch_size)

In [ ]:
def test_prediction(model, 
  test_dataloader,  
  device,
  ):
  
  test_predictions = []
  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      
      first_sen_len = data[3].to(device)
      second_sen_len = data[4].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      for i in output_cls:
        if i.item()<0:
          test_predictions.append(0)
        else:
          test_predictions.append(1)
      
      
     
     
      
  return test_predictions

In [ ]:
model_mpp.load_state_dict(torch.load('mpp_singe_value_state.bin', map_location=torch.device("cuda:0")))
model_ml.load_state_dict(torch.load('ml_singe_value_state.bin', map_location=torch.device("cuda:0")))


In [ ]:
test_mpp_predictions = test_prediction(model_mpp, 
  test_dataloader, 
  device
  )
test_ml_predictions = test_prediction(model_ml, 
  test_dataloader,
  device
  )

In [ ]:
df2['MPP_prediction'] = test_mpp_predictions
df2['ML_prediction'] = test_ml_predictions
df2.to_json('ERAI_pairwise_[aiml]_[1].json')